Period based on mass of central object:
$$T = 2\pi\sqrt{\frac{r^{3}}{GM}}$$
Lorentz factor for velocity:
$$\gamma_{velocity}=\frac{1}{\sqrt{1-(\frac{v}{c})^{2}}}$$
Lorentz factor for gravitational acceleration
$$\gamma_{gravity}=\frac{1}{\sqrt{1-\frac{2GM}{rc^{2}}}}$$

In [5]:
# don't forget to upload this to git
import numpy as np
import math

# Setting Constants
G          = 6.674e-11 # N * m^2 * kg^-2
c          = 2.99e+8   # m * s^-1
sol_mass   = 1.99e+30  # kg
earth_mass = 5.97e+24  # kg
moon_mass  = 7.348e+22 # kg
moon_dist  = 3.84e+8   # m
jup_mass   = 1.89e+27  # kg
bh_mass    = 1e8 * sol_mass # Intersteller gargantua = 1e8, TON618 = 6.6e10
au         = 1.49e+11  # m
year       = 3600*24*365.25 # in seconds
second     = 1
lightyear  = c * year



time_scales = [
    (1e9 * year, "billion year"),
    (1e6 * year, "million year"),
    (1e3 * year, "thousand year"),
    (year, "year"),
    (1, "second"),
    (1e-6, "microsecond"),
    (1e-9, "nanosecond"),
    (1e-12, "picosecond"),
    (5.39e-44, "Planck second")  
]


# Creating a function to calculate escape velocity
def calc_escape_velocity(mass, distance):
    V_esc = np.sqrt((2 * G * mass)/distance)
    return V_esc

# Creating a function to find the distances of chosen escape velocities
def calc_esc_radius(mass, V_esc):
    # Should give swarzchild radius when V_esc = c
    d = (2 * G * mass)/V_esc **2
    return d

def calc_radial_velocity(radius, period):
    rad_vel = (2 * np.pi * radius)/period
    return rad_vel

def calc_radius_from_vel(rad_vel, period):
    # r = vT/2*pi
    radius = (rad_vel * period)/ (2 * np.pi)
    return radius

def calc_period(distance, mass):
    period = 2 * np.pi *(math.sqrt(distance**3/(G * mass)))
    return period

def calc_grav_time_dilation(s_radius, distance):
    grav_gamma      = 1/(1 - (s_radius / distance))**0.5
    return grav_gamma

def calc_vel_time_dilation(velocity):
    if velocity >= 0.9999 * c:
        vel_gamma = 1/(math.sqrt(1-(0.9999/c)**2))
        return vel_gamma
    else:
        vel_gamma = 1/(math.sqrt(1-(velocity/c)**2))
        return vel_gamma

# kg/m^3
def calc_bh_density(mass, s_radius):
    volume = (4/3)*(np.pi * s_radius **3)
    density = mass / volume
    return density
    
moon_speed = calc_radial_velocity(moon_dist, 3600*24*27.32)
print(f"Moon velocity = {moon_speed:.2f}m/s")

s_radius        = calc_esc_radius(bh_mass, c)
if s_radius    <= 1e-3:
    print(f"Schwarzschild Radius = {s_radius:.2e}m")
else:
    print(f"Schwarzschild Radius = {s_radius:.2f}m or {s_radius/au:.2e}au")

bh_density     = calc_bh_density(bh_mass, s_radius)
print(f"Density of black hole = {bh_density:.3f}kg/m^3")

# Fails at 3 if mass = 5e10
n = 4 # shouldn't go below 1.5
orbit_n         =  n * s_radius
print(f"{n} time(s) the event horizon radius: {orbit_n:.2e}m, or {orbit_n/1000:.2e}km, or {orbit_n/au:.2e}au")

bh_period       = calc_period(orbit_n, bh_mass)
print(f"Period at {n} * event horizon: {bh_period:.2e}s or {bh_period/year:.2f}years.")

bh_orbit_speed  = calc_radial_velocity(orbit_n, bh_period)
if bh_orbit_speed >= c/10:
    print(f"Planet is orbiting with relativistic speeds at {bh_orbit_speed/c:.2f}c")
else:
    print(f"Speed of orbiting planet: {bh_orbit_speed:.2e}m/s or {bh_orbit_speed/c:.2f}c")
    
escape_v_n      = calc_escape_velocity(bh_mass, orbit_n)
print(f"Escape velocity at {n} schwarzchild radii: {escape_v_n:.2e}m/s or {escape_v_n/bh_orbit_speed:.2e} x radial speed or {escape_v_n/c:.2e}c.")

grav_gamma      = calc_grav_time_dilation(s_radius, orbit_n)
print(f"Gravitational gamma factor {grav_gamma}")

vel_gamma       = calc_vel_time_dilation(escape_v_n)
print(f"Escape Velocity gamma factor {vel_gamma:.2f}")

radial_gamma    = calc_vel_time_dilation(bh_orbit_speed)
print(f"Radial orbit velocity gamma factor: {radial_gamma:.2f}")

# If the math is working right, the photon sphere should have an orbital speed of c
orbit_c         = calc_radial_velocity(1.5*s_radius, calc_period(1.5*s_radius,bh_mass))
if orbit_c == c:
    print("The photon sphere is accurately at 1.5 * the event horizon radius.")
else:
    print(f"The photon sphere is still broken - orbits are showing as stable at {orbit_c/c}c")


Moon velocity = 1022.15m/s
Schwarzschild Radius = 297116587062.78m or 1.99e+00au
Density of black hole = 1811.272kg/m^3
4 time(s) the event horizon radius: 1.19e+12m, or 1.19e+09km, or 7.98e+00au
Period at 4 * event horizon: 7.06e+04s or 0.00years.
Planet is orbiting with relativistic speeds at 0.35c
Escape velocity at 4 schwarzchild radii: 1.50e+08m/s or 1.41e+00 x radial speed or 5.00e-01c.
Gravitational gamma factor 1.1547005383792517
Escape Velocity gamma factor 1.15
Radial orbit velocity gamma factor: 1.07
The photon sphere is still broken - orbits are showing as stable at 0.5773502691896258c


In [9]:
import pygame
import sys
import math
pygame.init()

# Creating a figure and axis
width, height = 700, 700
win = pygame.display.set_mode((width, height))
pygame.display.set_caption("Orbit Simulation")

# Color library
white        = (255,255,255)
black        = (0, 0, 0)
yellow       = (255, 255, 0)
blue         = (0, 0, 255)
red          = (255, 0, 0)
cosmic_latte = (255, 248, 231) # average color of galaxies
gray         = (222, 222, 222)
light_blue   = (204,212,255)
brown        = (77,38,0)

# Font
font         = pygame.font.SysFont("ariel", 16)
pause_bttn   = pygame.Rect(20, 20, 100, 50)

# Time and space scaling
# Still needs to handle whether or not a black hole is the central object
# For now, comment out the one not being used
dt            = bh_period/950
scale         = 150 / orbit_n

#dt            = year/360
#dt            = 250 / au

class Planet: 
    def __init__(self, x, y, radius, color, mass):
        self.x               = x
        self.y               = y
        self.radius          = radius # in pixels
        self.color           = color
        self.mass            = mass
        self.rel_clock       = 0 
        
        # Should also make light object functionality
        self.sun             = False
        self.distance_to_sun = 1 # Avoiding division by 0
        
        # To draw event horizons and handle collisions
        self.bh              = False
        
        self.orbit           = []
        
        self.x_vel           = 0 # m/s
        self.y_vel           = 0 # m/s
        self.speed_magnitude = math.sqrt(self.x_vel**2 + self.y_vel**2)
            
            
    def draw_event_horizon(self, planets):
        for planet in planets:
            if self.bh:
                center = (scale * self.x + (width/2), scale * self.y + (height/2))
                # Draw event horizon
                pygame.draw.circle(win, black, center, int(scale*s_radius), 1)
                # Draw photon sphere
                pygame.draw.circle(win, yellow, center, int(scale * 1.5 * s_radius),1)
    
    # This isn't working, I have no idea why not.    
    def distance_line(self, other_objects):
        self.position  = (self.x, self.y)
        for other in other_objects:
            other.position = (scale * other.x + (width/2), scale * other.y + (height/2))
            if other.sun or other.bh:
                pygame.draw.line(win, white, (self.position), (other.position), 1)

    
    def attraction(self, other):
        other_x, other_y = other.x, other.y
        distance_x       = other_x - self.x
        distance_y       = other_y - self.y
        distance         = math.sqrt(distance_x ** 2 + distance_y ** 2)
        
        if other.sun:
            self.distance_to_sun = distance
        elif other.bh:
            self.distance_to_sun = distance
        
        #mu      = self.mass * other.mass
        force   = G * self.mass * other.mass / distance ** 2
        theta   = math.atan2(distance_y, distance_x)
        force_x = math.cos(theta) * force
        force_y = math.sin(theta) * force
        return force_x, force_y
    
    def update_position(self, planets):
        total_fx = total_fy = 0
        for planet in planets:
            # Avoid division by 0
            if self == planet:
                continue
            fx, fy    = self.attraction(planet)
            total_fx += fx
            total_fy += fy
            
        # Calculate acceleration
        accel_x = total_fx / self.mass
        accel_y = total_fy / self.mass
    
        # Update velocity components
        self.x_vel += accel_x * dt
        self.y_vel += accel_y * dt
        
        self.x += self.x_vel * dt
        self.y += self.y_vel * dt
        
        self.orbit.append((self.x, self.y))
        
        self.speed_magnitude = math.sqrt(self.x_vel**2 + self.y_vel**2)
        if self.speed_magnitude >= c:
            # Cap velocity
            self.x_vel = self.x_vel / self.speed_magnitude * (c * 0.9999)
            self.y_vel = self.y_vel / self.speed_magnitude * (c * 0.9999)
    
    # Have things fall into the black hole
    #def collide(self, other):
    #    if not self.bh or self.sun:
    #        if other.bh:
    #            if self.distance_to_bh <= s_radius:
    #               other.mass += self.mass
    #                setattr(self, "absorbed", True)
    
    def draw(self, win):
        x           = self.x * scale + width/2
        y           = self.y * scale + height/2
        
        
        if len(self.orbit) > 2:
            # Keep only the most recent 1000 points
            if len(self.orbit) > 30000:
                self.orbit = self.orbit[-30000:]
            
            updated_points = []
            
            for point in self.orbit:
                x, y = point
                x = x * scale + width/2
                y = y * scale + height/2
                updated_points.append((x,y))

            pygame.draw.lines(win, self.color, False, updated_points, 1)
        
            
        pygame.draw.circle(win, self.color, (x,y), self.radius)
        
        if not self.sun:
            if not self.bh:
                grav_gamma          = calc_grav_time_dilation(s_radius, self.distance_to_sun)
                
                grav_gamma_text     = font.render(f"Gγ: {grav_gamma:.3f}", 1, self.color)
                
                if self.speed_magnitude <= 0.9999*c:
                    self.vel_gamma  = calc_vel_time_dilation(math.sqrt(self.x_vel**2 + self.y_vel**2))
                else:
                    self.vel_gamma  = 70.1
                    
                vel_gamma_text      = font.render(f"Vγ: {self.vel_gamma:.3f}", 1, self.color)
                
                # Update inertial and relative time counters
                self.rel_clock     += self.vel_gamma * grav_gamma * dt
                #self.intl_clock    += dt
                
                
                # Change the time scale as the counters gets higher
                for exponent, unit in time_scales:
                    # Check if time_counter or relative_time is greater than the current scale
                    if self.rel_clock >= exponent:
                        # Use the current scale for rendering
                        if unit == "Planck second":
                            #intl_time_text = font.render(f"Inertial time in {unit}s: {self.intl_clock:.3e}", 1, self.color)
                            rel_time_text  = font.render(f"Time in {unit}s: {self.rel_clock/exponent:.3e}", 1, self.color)
                            #dif_time_text = font.render(f"Difference between clocks in {unit}s: {time_diff/exponent:.3e} {unit}", 1, black)
                        else:
                            #intl_time_text = font.render(f"Inertial time in {unit}s: {self.intl_clock/exponent:.3f}", 1, self.color)
                            rel_time_text  = font.render(f"Clock on planet in {unit}s: {self.rel_clock/exponent:.3f}", 1, self.color)
                            #dif_time_text = font.render(f"Difference between clocks in {unit}s: {time_diff/exponent:.3f}", 1, black)
                        break
                
                print(f"Debugging: dt: {dt:.3f} Vγ: {self.vel_gamma:.3f}, Gγ: {grav_gamma:.3f}, Rel_clock: {self.rel_clock:.5f})")#, Inertial clock: {self.intl_clock:.5f}")
                
                if self.distance_to_sun >= lightyear:
                    distance_text = font.render(f"{(self.distance_to_sun-s_radius)/lightyear:.2f}lyrs from EH", 1, self.color)
                elif self.distance_to_sun >= au/1000:
                    distance_text = font.render(f"{(self.distance_to_sun-s_radius)/au:.3e}au from EH", 1, self.color)
                else:
                    distance_text = font.render(f"{(self.distance_to_sun-s_radius):.3e}m from EH", 1, self.color)
                    
            win.blit(distance_text, (x - distance_text.get_width()/2, y - distance_text.get_height()/2))
            win.blit(grav_gamma_text, (x + 10 - grav_gamma_text.get_width()/2, y + 10 - grav_gamma_text.get_width()/2))
            win.blit(vel_gamma_text, (x +2 - vel_gamma_text.get_width()/2, y - 1 - vel_gamma_text.get_width()/2))
            win.blit(rel_time_text, (x - rel_time_text.get_width()/2, y - rel_time_text.get_width()/2))
             
def main():
    run            = True
    paused         = False
    clock          = pygame.time.Clock()
    time_counter   = 0
    relative_time  = 0
    time_diff      = 0
    
    sun            = Planet(0,0, 9, yellow, sol_mass)
    sun.sun        = True
    
    black_hole     = Planet(0,0,3, black, bh_mass)
    black_hole.sun = True
    black_hole.bh  = True
    
    mercury        = Planet(0.39 * au, 0, 2, gray, 0.33e+24)
    mercury.y_vel  = -calc_radial_velocity(0.39*au, 0.24*year)
    
    venus          = Planet(0.72 * au, 0, 5, red, 4.87e+24)
    venus.y_vel    = -calc_radial_velocity(0.72*au, 0.62*year)
    
    earth          = Planet(au, 0, 2, blue, earth_mass)
    earth.y_vel    = -calc_radial_velocity(au, year)
    
    moon           = Planet(au + moon_dist, 0, 2, gray, moon_mass)
    moon.y_vel     = -(moon_speed-earth.y_vel)
    
    mars           = Planet(1.524 * au, 0, 4, red, 0.11 * earth_mass)
    mars.y_vel     = -2.41e4
    
    jupiter        = Planet(-5.2 * au, 0, 8, red, 0.01*sol_mass)#jup_mass)
    jupiter.y_vel  = calc_radial_velocity(5.2 * au, 12* year)
    
    saturn         = Planet(9.54 * au,0, 5, yellow, 568e+24)
    saturn.y_vel   = -calc_radial_velocity(9.54 * au, 28.7*year)
    
    light          = Planet(1.5 * s_radius,0, 1, white, 1)
    light.y_vel    = -np.sin(c)
    light.x_vel    = -np.cos(c)
    
    t_plnt         = Planet(-orbit_n, 0, 2, black, 0.001*bh_mass)
    t_plnt.y_vel   = -0.7*escape_v_n
    
    # Make the differences from t_plnt a single variable
    t2 = 1.5
    t2_plnt        = Planet(t2 * orbit_n, 0, 2, red, 0.002*bh_mass)
    t2_plnt.y_vel  = 0.8*calc_radial_velocity(t2 * orbit_n, calc_period(t2*orbit_n, bh_mass + t_plnt.mass))
    
    t3 = 2
    t3_plnt        = Planet(-t3 * orbit_n, 0, 2, blue, 0.003*bh_mass)
    t3_plnt.y_vel  = -0.5*calc_radial_velocity(t3 * orbit_n, calc_period(t3*orbit_n, bh_mass + t_plnt.mass + t2_plnt.mass))
    
    planets = [black_hole, t3_plnt]
    
    while run:
        clock.tick(60)
        win.fill(light_blue)
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if event.button == 1:
                    if pause_bttn.collidepoint(event.pos):
                        paused = not paused
        if not paused:

            # Update time counter
            #time_counter       += dt
            #relative_time      += vel_gamma * grav_gamma * dt
            #time_diff           = relative_time - time_counter


            if t_plnt.speed_magnitude <= c/1000:
                vel_text        = font.render(f"Inertial frame velocity of planet: {t_plnt.speed_magnitude:.3f}m/s",1, black)
            else:
                vel_text        = font.render(f"Inertial frame velocity of planet: {t_plnt.speed_magnitude/c:.3f}c",1, black)

            # Change the time scale as the counters gets higher
            #for exponent, unit in time_scales:
                # Check if time_counter or relative_time is greater than the current scale
            #    if time_counter >= exponent:
                    # Use the current scale for rendering
            #        if unit == "Planck second":
            #            time_text     = font.render(f"Inertial Clock in {unit}s: {time_counter/exponent:.3e}", 1, black)
            #            rel_time_text = font.render(f"Clock on planet in {unit}s: {relative_time/exponent:.3e}", 1, black)
            #            dif_time_text = font.render(f"Difference between clocks in {unit}s: {time_diff/exponent:.3e} {unit}", 1, black)
            #        else:
            #            time_text = font.render(f"Inertial Clock in {unit}s: {time_counter/exponent:.3f}", 1, black)
            #            rel_time_text = font.render(f"Clock on planet in {unit}s: {relative_time/exponent:.3f}", 1, black)
            #            dif_time_text = font.render(f"Difference between clocks in {unit}s: {time_diff/exponent:.3f}", 1, black)
            #        break



            # Print relevant values
            #win.blit(time_text, (10, 10)) 
            #win.blit(rel_time_text, (10, 30))
            #win.blit(dif_time_text, (10, 50))
            win.blit(vel_text, (10, 70))

            for planet in planets:
                planet.update_position(planets)
                planet.draw(win)
                #planet.distance_line(planets)
                planet.draw_event_horizon(planets)
    

            
        pygame.display.update()
                
    pygame.quit
    
    
main()

Debugging: dt: 74.356 Vγ: 1.008, Gγ: 1.069, Rel_clock: 80.12039)
Debugging: dt: 74.356 Vγ: 1.008, Gγ: 1.069, Rel_clock: 160.24084)
Debugging: dt: 74.356 Vγ: 1.008, Gγ: 1.069, Rel_clock: 240.36140)
Debugging: dt: 74.356 Vγ: 1.008, Gγ: 1.069, Rel_clock: 320.48212)
Debugging: dt: 74.356 Vγ: 1.008, Gγ: 1.069, Rel_clock: 400.60302)
Debugging: dt: 74.356 Vγ: 1.008, Gγ: 1.069, Rel_clock: 480.72417)
Debugging: dt: 74.356 Vγ: 1.008, Gγ: 1.069, Rel_clock: 560.84560)
Debugging: dt: 74.356 Vγ: 1.008, Gγ: 1.069, Rel_clock: 640.96736)
Debugging: dt: 74.356 Vγ: 1.008, Gγ: 1.069, Rel_clock: 721.08949)
Debugging: dt: 74.356 Vγ: 1.008, Gγ: 1.069, Rel_clock: 801.21204)
Debugging: dt: 74.356 Vγ: 1.008, Gγ: 1.069, Rel_clock: 881.33505)
Debugging: dt: 74.356 Vγ: 1.008, Gγ: 1.069, Rel_clock: 961.45857)
Debugging: dt: 74.356 Vγ: 1.008, Gγ: 1.069, Rel_clock: 1041.58264)
Debugging: dt: 74.356 Vγ: 1.008, Gγ: 1.069, Rel_clock: 1121.70731)
Debugging: dt: 74.356 Vγ: 1.008, Gγ: 1.069, Rel_clock: 1201.83261)
Debuggin

Debugging: dt: 74.356 Vγ: 1.010, Gγ: 1.072, Rel_clock: 10110.53705)
Debugging: dt: 74.356 Vγ: 1.010, Gγ: 1.072, Rel_clock: 10191.03238)
Debugging: dt: 74.356 Vγ: 1.010, Gγ: 1.072, Rel_clock: 10271.53393)
Debugging: dt: 74.356 Vγ: 1.010, Gγ: 1.072, Rel_clock: 10352.04178)
Debugging: dt: 74.356 Vγ: 1.010, Gγ: 1.072, Rel_clock: 10432.55597)
Debugging: dt: 74.356 Vγ: 1.010, Gγ: 1.072, Rel_clock: 10513.07656)
Debugging: dt: 74.356 Vγ: 1.010, Gγ: 1.072, Rel_clock: 10593.60363)
Debugging: dt: 74.356 Vγ: 1.010, Gγ: 1.072, Rel_clock: 10674.13722)
Debugging: dt: 74.356 Vγ: 1.010, Gγ: 1.072, Rel_clock: 10754.67739)
Debugging: dt: 74.356 Vγ: 1.010, Gγ: 1.072, Rel_clock: 10835.22421)
Debugging: dt: 74.356 Vγ: 1.010, Gγ: 1.072, Rel_clock: 10915.77775)
Debugging: dt: 74.356 Vγ: 1.011, Gγ: 1.072, Rel_clock: 10996.33804)
Debugging: dt: 74.356 Vγ: 1.011, Gγ: 1.072, Rel_clock: 11076.90517)
Debugging: dt: 74.356 Vγ: 1.011, Gγ: 1.072, Rel_clock: 11157.47919)
Debugging: dt: 74.356 Vγ: 1.011, Gγ: 1.072, Rel_

Debugging: dt: 74.356 Vγ: 1.018, Gγ: 1.081, Rel_clock: 20079.13923)
Debugging: dt: 74.356 Vγ: 1.018, Gγ: 1.081, Rel_clock: 20160.95588)
Debugging: dt: 74.356 Vγ: 1.018, Gγ: 1.081, Rel_clock: 20242.78923)
Debugging: dt: 74.356 Vγ: 1.018, Gγ: 1.081, Rel_clock: 20324.63941)
Debugging: dt: 74.356 Vγ: 1.018, Gγ: 1.081, Rel_clock: 20406.50657)
Debugging: dt: 74.356 Vγ: 1.018, Gγ: 1.081, Rel_clock: 20488.39081)
Debugging: dt: 74.356 Vγ: 1.018, Gγ: 1.082, Rel_clock: 20570.29229)
Debugging: dt: 74.356 Vγ: 1.019, Gγ: 1.082, Rel_clock: 20652.21113)
Debugging: dt: 74.356 Vγ: 1.019, Gγ: 1.082, Rel_clock: 20734.14746)
Debugging: dt: 74.356 Vγ: 1.019, Gγ: 1.082, Rel_clock: 20816.10143)
Debugging: dt: 74.356 Vγ: 1.019, Gγ: 1.082, Rel_clock: 20898.07316)
Debugging: dt: 74.356 Vγ: 1.019, Gγ: 1.082, Rel_clock: 20980.06279)
Debugging: dt: 74.356 Vγ: 1.019, Gγ: 1.082, Rel_clock: 21062.07047)
Debugging: dt: 74.356 Vγ: 1.019, Gγ: 1.082, Rel_clock: 21144.09633)
Debugging: dt: 74.356 Vγ: 1.019, Gγ: 1.083, Rel_

Debugging: dt: 74.356 Vγ: 1.038, Gγ: 1.105, Rel_clock: 30235.90652)
Debugging: dt: 74.356 Vγ: 1.038, Gγ: 1.105, Rel_clock: 30321.23204)
Debugging: dt: 74.356 Vγ: 1.038, Gγ: 1.106, Rel_clock: 30406.60577)
Debugging: dt: 74.356 Vγ: 1.039, Gγ: 1.106, Rel_clock: 30492.02823)
Debugging: dt: 74.356 Vγ: 1.039, Gγ: 1.106, Rel_clock: 30577.49992)
Debugging: dt: 74.356 Vγ: 1.039, Gγ: 1.107, Rel_clock: 30663.02138)
Debugging: dt: 74.356 Vγ: 1.040, Gγ: 1.107, Rel_clock: 30748.59315)
Debugging: dt: 74.356 Vγ: 1.040, Gγ: 1.107, Rel_clock: 30834.21576)
Debugging: dt: 74.356 Vγ: 1.040, Gγ: 1.108, Rel_clock: 30919.88976)
Debugging: dt: 74.356 Vγ: 1.040, Gγ: 1.108, Rel_clock: 31005.61571)
Debugging: dt: 74.356 Vγ: 1.041, Gγ: 1.108, Rel_clock: 31091.39420)
Debugging: dt: 74.356 Vγ: 1.041, Gγ: 1.109, Rel_clock: 31177.22578)
Debugging: dt: 74.356 Vγ: 1.041, Gγ: 1.109, Rel_clock: 31263.11105)
Debugging: dt: 74.356 Vγ: 1.042, Gγ: 1.110, Rel_clock: 31349.05060)
Debugging: dt: 74.356 Vγ: 1.042, Gγ: 1.110, Rel_

Debugging: dt: 74.356 Vγ: 1.121, Gγ: 1.207, Rel_clock: 41289.69424)
Debugging: dt: 74.356 Vγ: 1.123, Gγ: 1.209, Rel_clock: 41390.59307)
Debugging: dt: 74.356 Vγ: 1.124, Gγ: 1.211, Rel_clock: 41491.83161)
Debugging: dt: 74.356 Vγ: 1.126, Gγ: 1.213, Rel_clock: 41593.41904)
Debugging: dt: 74.356 Vγ: 1.128, Gγ: 1.215, Rel_clock: 41695.36493)
Debugging: dt: 74.356 Vγ: 1.130, Gγ: 1.218, Rel_clock: 41797.67923)
Debugging: dt: 74.356 Vγ: 1.132, Gγ: 1.220, Rel_clock: 41900.37229)
Debugging: dt: 74.356 Vγ: 1.134, Gγ: 1.223, Rel_clock: 42003.45493)
Debugging: dt: 74.356 Vγ: 1.136, Gγ: 1.225, Rel_clock: 42106.93839)
Debugging: dt: 74.356 Vγ: 1.138, Gγ: 1.228, Rel_clock: 42210.83442)
Debugging: dt: 74.356 Vγ: 1.140, Gγ: 1.230, Rel_clock: 42315.15528)
Debugging: dt: 74.356 Vγ: 1.142, Gγ: 1.233, Rel_clock: 42419.91374)
Debugging: dt: 74.356 Vγ: 1.145, Gγ: 1.236, Rel_clock: 42525.12319)
Debugging: dt: 74.356 Vγ: 1.147, Gγ: 1.239, Rel_clock: 42630.79761)
Debugging: dt: 74.356 Vγ: 1.149, Gγ: 1.242, Rel_

Debugging: dt: 74.356 Vγ: 1.312, Gγ: 1.466, Rel_clock: 66041.84833)
Debugging: dt: 74.356 Vγ: 1.304, Gγ: 1.454, Rel_clock: 66182.76129)
Debugging: dt: 74.356 Vγ: 1.296, Gγ: 1.442, Rel_clock: 66321.66082)
Debugging: dt: 74.356 Vγ: 1.288, Gγ: 1.431, Rel_clock: 66458.65749)
Debugging: dt: 74.356 Vγ: 1.280, Gγ: 1.420, Rel_clock: 66593.85373)
Debugging: dt: 74.356 Vγ: 1.273, Gγ: 1.410, Rel_clock: 66727.34454)
Debugging: dt: 74.356 Vγ: 1.266, Gγ: 1.401, Rel_clock: 66859.21813)
Debugging: dt: 74.356 Vγ: 1.259, Gγ: 1.392, Rel_clock: 66989.55650)
Debugging: dt: 74.356 Vγ: 1.253, Gγ: 1.383, Rel_clock: 67118.43598)
Debugging: dt: 74.356 Vγ: 1.247, Gγ: 1.375, Rel_clock: 67245.92772)
Debugging: dt: 74.356 Vγ: 1.241, Gγ: 1.367, Rel_clock: 67372.09809)
Debugging: dt: 74.356 Vγ: 1.236, Gγ: 1.359, Rel_clock: 67497.00910)
Debugging: dt: 74.356 Vγ: 1.230, Gγ: 1.352, Rel_clock: 67620.71875)
Debugging: dt: 74.356 Vγ: 1.225, Gγ: 1.345, Rel_clock: 67743.28131)
Debugging: dt: 74.356 Vγ: 1.220, Gγ: 1.339, Rel_

Debugging: dt: 74.356 Vγ: 1.054, Gγ: 1.126, Rel_clock: 78639.90091)
Debugging: dt: 74.356 Vγ: 1.054, Gγ: 1.125, Rel_clock: 78728.05211)
Debugging: dt: 74.356 Vγ: 1.053, Gγ: 1.124, Rel_clock: 78816.12228)
Debugging: dt: 74.356 Vγ: 1.053, Gγ: 1.124, Rel_clock: 78904.11248)
Debugging: dt: 74.356 Vγ: 1.052, Gγ: 1.123, Rel_clock: 78992.02377)
Debugging: dt: 74.356 Vγ: 1.052, Gγ: 1.123, Rel_clock: 79079.85716)
Debugging: dt: 74.356 Vγ: 1.052, Gγ: 1.122, Rel_clock: 79167.61367)
Debugging: dt: 74.356 Vγ: 1.051, Gγ: 1.122, Rel_clock: 79255.29429)
Debugging: dt: 74.356 Vγ: 1.051, Gγ: 1.121, Rel_clock: 79342.89997)
Debugging: dt: 74.356 Vγ: 1.050, Gγ: 1.121, Rel_clock: 79430.43169)
Debugging: dt: 74.356 Vγ: 1.050, Gγ: 1.120, Rel_clock: 79517.89036)
Debugging: dt: 74.356 Vγ: 1.050, Gγ: 1.120, Rel_clock: 79605.27691)
Debugging: dt: 74.356 Vγ: 1.049, Gγ: 1.119, Rel_clock: 79692.59223)
Debugging: dt: 74.356 Vγ: 1.049, Gγ: 1.119, Rel_clock: 79779.83721)
Debugging: dt: 74.356 Vγ: 1.048, Gγ: 1.118, Rel_

Debugging: dt: 74.356 Vγ: 1.023, Gγ: 1.088, Rel_clock: 88998.62417)
Debugging: dt: 74.356 Vγ: 1.023, Gγ: 1.087, Rel_clock: 89081.34308)
Debugging: dt: 74.356 Vγ: 1.023, Gγ: 1.087, Rel_clock: 89164.03838)
Debugging: dt: 74.356 Vγ: 1.023, Gγ: 1.087, Rel_clock: 89246.71025)
Debugging: dt: 74.356 Vγ: 1.023, Gγ: 1.087, Rel_clock: 89329.35889)
Debugging: dt: 74.356 Vγ: 1.022, Gγ: 1.087, Rel_clock: 89411.98448)
Debugging: dt: 74.356 Vγ: 1.022, Gγ: 1.087, Rel_clock: 89494.58721)
Debugging: dt: 74.356 Vγ: 1.022, Gγ: 1.086, Rel_clock: 89577.16727)
Debugging: dt: 74.356 Vγ: 1.022, Gγ: 1.086, Rel_clock: 89659.72483)
Debugging: dt: 74.356 Vγ: 1.022, Gγ: 1.086, Rel_clock: 89742.26008)
Debugging: dt: 74.356 Vγ: 1.022, Gγ: 1.086, Rel_clock: 89824.77320)
Debugging: dt: 74.356 Vγ: 1.022, Gγ: 1.086, Rel_clock: 89907.26436)
Debugging: dt: 74.356 Vγ: 1.022, Gγ: 1.086, Rel_clock: 89989.73374)
Debugging: dt: 74.356 Vγ: 1.021, Gγ: 1.086, Rel_clock: 90072.18151)
Debugging: dt: 74.356 Vγ: 1.021, Gγ: 1.085, Rel_

SystemExit: 